In [1]:
# Import packages

import boto3
import pandas as pd
import io
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
# Reading the data into a DataFrame

s3 = boto3.client('s3')

bucket_name = 'tech-challenge-t3'
folder_name = 'raw/'

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)
 
dataFrames = []

if 'Contents' in response:
    for obj in response['Contents']:
        file_key = obj['Key']
        if file_key.endswith('.parquet'):
            print(f"Loading file: {file_key}")
            
            file_obj = s3.get_object(Bucket=bucket_name, Key=file_key)
            file_content = file_obj['Body'].read()

            df = pd.read_parquet(io.BytesIO(file_content))
            dataFrames.append(df)

transactions = pd.concat(dataFrames, ignore_index=True)

transactions.head()

Loading file: raw/transacoes_2022_1001_1500.parquet
Loading file: raw/transacoes_2022_1501_2000.parquet
Loading file: raw/transacoes_2022_1_500.parquet
Loading file: raw/transacoes_2022_2001_2500.parquet
Loading file: raw/transacoes_2022_2501_2769.parquet
Loading file: raw/transacoes_2022_501_1000.parquet
Loading file: raw/transacoes_2023_1201_1500.parquet
Loading file: raw/transacoes_2023_1501_2000.parquet
Loading file: raw/transacoes_2023_1_300.parquet
Loading file: raw/transacoes_2023_2001_2500.parquet
Loading file: raw/transacoes_2023_2501_3034.parquet
Loading file: raw/transacoes_2023_301_600.parquet
Loading file: raw/transacoes_2023_601_900.parquet
Loading file: raw/transacoes_2023_901_1200.parquet
Loading file: raw/transacoes_2024.parquet


,codEmpresa,nomEmpresa,numReserva,codEmpenho,anoEmpenho,mesEmpenho,datEmpenho,codProcesso,numCpfCnpj,txtRazaoSocial,...,valTotalEmpenhado,valAnuladoEmpenho,valEmpenhadoLiquido,valLiquidado,valPagoExercicio,valPagoRestos,codReferencia,codDestinacaoRecurso,codVinculacaoRecurso,codExeFonte
0,01,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,32266,44273,2022,5,18/05/2022 00:00:00,6018201800027804,12183911000180,ALAVANCA SAUDE E APOIO DOMICILIAR LTDA,...,5244.00,0.0,5244.00,5244.00,5244.00,0.0,,,,
1,01,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,32265,44274,2022,5,18/05/2022 00:00:00,6110202200049759,47411780000126,BACE COMERCIO INTERNACIONAL LTDA.,...,123504.00,961.0,122543.00,122543.00,122543.00,0.0,,,,
2,01,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,16855,44275,2022,5,18/05/2022 00:00:00,6020201800008738,01632005000108,JANSIL LOCADORA DE VEICULOS E TRANSPORTES LTDA...,...,6653.90,0.0,6653.90,6653.90,6653.90,0.0,,,,
3,01,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,16847,44276,2022,5,18/05/2022 00:00:00,6020201800033678,17625812804,JOEL GAUDENCIO DE SOUZA,...,11505.32,0.0,11505.32,11505.32,11505.32,0.0,,,,
4,01,PREFEITURA DO MUNICÍPIO DE SÃO PAULO,16855,44277,2022,5,18/05/2022 00:00:00,6020201800008061,12684713000109,SANTIAGO & ALTEMAR TRANSPORTES LTDA ME,...,7355.18,0.0,7355.18,7355.18,7355.18,0.0,,,,


In [3]:
# Selecting specific features

features = transactions[
    [
        "codEmpresa",
        "anoEmpenho",
        "mesEmpenho",
        "numCpfCnpj",
        "valPagoExercicio",
        "valPagoRestos",
        "codOrgao",
        "codSubFuncao",
        "codPrograma",
        "codProjetoAtividade",
        "codCategoria",
        "codGrupo",
        "codModalidade",
        "codElemento",
        "codFonteRecurso",
        "codItemDespesa",
        "codSubElemento"
    ]
]

In [21]:
# Pre processing the features

processed_features = features.dropna()
processed_features = processed_features[processed_features['valPagoExercicio'] != 0]

processed_features["valTotal"] =  processed_features["valPagoExercicio"] + processed_features["valPagoRestos"]
processed_features = processed_features.drop(columns=['valPagoExercicio', 'valPagoRestos'])

le = LabelEncoder()

processed_features["numCpfCnpj"] = le.fit_transform(processed_features["numCpfCnpj"])

processed_features["codFonteRecurso"] = le.fit_transform(processed_features["codFonteRecurso"])

columns_to_be_converted = [
    "codEmpresa",
    "anoEmpenho",
    "mesEmpenho",
    "codOrgao",
    "codSubFuncao",
    "codPrograma",
    "codProjetoAtividade",
    "codCategoria",
    "codGrupo",
    "codModalidade",
    "codElemento",
    "codItemDespesa",
    "codSubElemento"
]


for column in columns_to_be_converted:
    processed_features[column] = processed_features[column].astype(int)


scaler = StandardScaler()
standardized_features = scaler.fit_transform(processed_features)



In [14]:
# Running the DBSCAN algorithm

dbscan = DBSCAN(eps=0.5, min_samples=2, metric='euclidean', algorithm='auto')
labels = dbscan.fit_predict(standardized_features)
processed_features['cluster'] = labels

transactions['cluster'] = processed_features['cluster'].reindex(transactions.index)

In [20]:
# Filtering the anomalies

anomalies = transactions[transactions['cluster'] == -1]

print(anomalies.head())

print(f"Number of anomalies detected: {len(anomalies)}")

    codEmpresa                                         nomEmpresa  numReserva  \
213         83  COMPANHIA METROPOLITANA DE HABITAÇÃO DE SÃO PAULO         223   
419         01               PREFEITURA DO MUNICÍPIO DE SÃO PAULO       32516   
425         06        SERVIÇO FUNERÁRIO DO MUNICÍPIO DE SÃO PAULO         269   
555         01               PREFEITURA DO MUNICÍPIO DE SÃO PAULO       32745   
560         01               PREFEITURA DO MUNICÍPIO DE SÃO PAULO       24105   

     codEmpenho  anoEmpenho  mesEmpenho           datEmpenho  \
213         241        2022           5  18/05/2022 00:00:00   
419       44649        2022           5  18/05/2022 00:00:00   
425         435        2022           5  18/05/2022 00:00:00   
555       44777        2022           5  18/05/2022 00:00:00   
560       44781        2022           5  18/05/2022 00:00:00   

          codProcesso      numCpfCnpj  \
213  7610202200011434  18394434000189   
419  6018202200357100  03590952000109   
425  

In [18]:
output_file = 'anomalies.parquet'
anomalies.to_parquet(output_file)

s3.upload_file(output_file, bucket_name, 'output/anomalies.parquet')
print(f"Anomalies saved to s3://{bucket_name}/output/anomalies.parquet")

Anomalies saved to s3://tech-challenge-t3/output/anomalies.parquet
